# Analyze Price Data

In [26]:
from datetime import datetime
import os
import pandas as pd

# Get Dates

In [21]:
def get_dates(root_folder):
    return sorted([datetime.strptime(file.split("_")[1].split(".")[0], '%d-%m-%Y') 
            for file in files if ".csv" in file])

In [22]:
root_folder = "raw_data"
dates = get_dates(root_folder)

# Get Data

In [41]:
def get_data(dates):
    data_list = [pd.read_csv("raw_data/data_{0}.csv".format(date.strftime('%d-%m-%Y'))) for date in dates]
    return pd.concat(data_list)

In [42]:
final_data = get_data(dates)

In [43]:
final_data

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
0,Cereales,Trigo panificable,"0,00 / 0,00","147,00","147,00",Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
2,Cereales,Avena,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
3,Cereales,Centeno,"0,00 / 0,00","139,00","139,00",Eur / Tm.,NaN,02-01-2006
4,Cereales,Maíz,"0,00 / 0,00","142,00","142,00",Eur / Tm.,NaN,02-01-2006
5,Leguminosas,Garbanzo pedrosillano,"0,00 / 0,00","600,00","600,00",Eur / Tm.,NaN,02-01-2006
6,Leguminosas,Lenteja de Salamanca,"0,00 / 0,00","0,00","0,00",Eur / Tm.,NaN,02-01-2006
7,Leguminosas,Lenteja armuña D.O.,"0,00 / 0,00","0,00","0,00",Eur / Tm.,NaN,02-01-2006
8,Forrajes,Forrajes (Avena),"0,00 / 0,00","0,00","0,00",Eur / Tm.,NaN,02-01-2006
9,Forrajes,Alfalfa empacada,"0,00 / 0,00","0,00","0,00",Eur / Tm.,NaN,02-01-2006


In [44]:
final_data[final_data["Categoría"]=="Cebada"]

,Producto,Categoría,Anterior,Último,Diferencia,Medida,Tendencia,Fecha
1,Cereales,Cebada,"0,00 / 0,00","145,00","145,00",Eur / Tm.,NaN,02-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,09-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,16-01-2006
1,Cereales,Cebada,"145,00","145,00","0,00",Eur / Tm.,NaN,23-01-2006
1,Cereales,Cebada,"145,00","144,00","-1,00",Eur / Tm.,NaN,30-01-2006
1,Cereales,Cebada,"144,00","143,00","-1,00",Eur / Tm.,NaN,06-02-2006
1,Cereales,Cebada,"143,00","142,00","-1,00",Eur / Tm.,NaN,13-02-2006
1,Cereales,Cebada,"142,00","141,00","-1,00",Eur / Tm.,NaN,20-02-2006
2,Cereales,Cebada,"141,00","140,00","-1,00",Eur / Tm.,NaN,27-02-2006
2,Cereales,Cebada,"140,00","138,00","-2,00",Eur / Tm.,NaN,06-03-2006
